In [1]:
#setup based on this: https://t-redactyl.io/blog/2020/08/reading-s3-data-into-a-spark-dataframe-using-sagemaker.html
import boto3
import json 
import time
import ntpath
import pandas as pd
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import sagemaker_pyspark
import botocore.session


## Set Spark Session Configuration

In [2]:
session = botocore.session.get_session()
credentials = session.get_credentials()

In [3]:
client = boto3.client('secretsmanager')
response = client.get_secret_value(
    SecretId='sapient-s3-access'
)
response = json.loads(response['SecretString'])
access_key = response["aws_access_key_id"]
secret_key = response["aws_secret_access_key"]

In [4]:
conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars()))
       )

In [5]:
# https://spark.apache.org/docs/latest/configuration.html#memory-management
spark = (
    SparkSession
    .builder
    .config(conf=conf) \
    .config('fs.s3a.access.key', access_key)
    .config('fs.s3a.secret.key', secret_key)
    .config('spark.network.timeout', 300)
    .config('spark.local.dir', '/home/ec2-user/SageMaker/tmp')
    .config("spark.executor.memory", "100g")
    .config("spark.driver.memory", "90g")
    .config('spark.sql.shuffle.partitions', 300)
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size","20g")
    .appName("sapient")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/25 15:11:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/25 15:11:22 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/25 15:11:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/25 15:11:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Functions to Load and Read Data

In [6]:
# read from raw bucket + write to refined bucket + aggregate final to the trusted bucket
s3_url_raw = "s3a://sapient-bucket-raw/"
s3_url_refined = "s3a://sapient-bucket-refined/"
s3_url_trusted = "s3a://sapient-bucket-trusted/"
ecar_cols = [
    'id','timestamp','objectID','actorID','object','action','hostname', 'user_name', 'privileges', 'image_path', 
    'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user'
            ]
bro_cols_conn = ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 
                 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents']
bro_cols_rep = ['ts', 'level', 'message', 'location']

In [7]:
# # Create a new dataframe with distinct objectIDs to identify malcious ObjectIds
df_labels = spark.read.parquet(f"{s3_url_refined}/prod/labels").cache()
malcious_events = list(df_labels.select('id').distinct().toPandas()['id'])
df_labels.unpersist()

DataFrame[hostname: string, id: string, objectID: string, actorID: string, timestamp: timestamp, object: string, action: string]

In [8]:
def getFirstEvents(df):
    """
    this creates then drops duplicates and gets the first appearance of each relationship entry
    input - dataframe with columns objectID and actorID
    output dataframe
    """
    window = Window.partitionBy("relationship").orderBy("timestamp")
    # label malicious events, filter to only hosts with malicious events
    df = df.withColumn('relationship', concat(df.actorID, lit('->'),df.objectID) ) \
                .withColumn('rank', rank().over(window)) \
                .filter(col('rank') == 1) \
                .drop('rank')
    mal_hosts = list(df.select('hostname').filter( col('malicious') == 1 ).distinct().toPandas()['hostname'])
    df = df.filter(col("hostname").isin(mal_hosts))
    return df

In [9]:
def readCheckpoint(type='ecar', env='prod', size='small'):
    """
    type: ecar, ecar-bro, bro
    """
    if type == 'labels':
        s3_parquet_loc = f"{s3_url_trusted}/{env}/{type}"
    else:
        s3_parquet_loc = f"{s3_url_trusted}/{env}/{type}/{size}"
    start_time = time.time()
    df = spark.read.parquet(s3_parquet_loc)\
                .withColumn("malicious", when(col('id').isin( malcious_events ), 1) 
                .otherwise(0))
    df.groupBy('malicious').count().show()
    df = df.filter( col('actorID') != col('objectID')) 
    # get first connect events
    df = getFirstEvents(df)
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and cache time: %s seconds ---" % (time.time() - start_time))
    return df

In [10]:
def write_firsts(size='all'):
    """
    this creates then drops duplicates and writes them to file in S3
    """
    start_time = time.time()
    df = readCheckpoint(size=size).cache()
    # create file features
    df.withColumn("image_path", getFileUDF(col("image_path"))) \
        .withColumn("parent_image_path", getFileUDF(col("parent_image_path"))) \
        .withColumn("new_path", getFileUDF(col("new_path"))) \
        .withColumn("file_path", getFileUDF(col("file_path"))) \
        .withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0))
    df_first_events.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/first_events")
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and write time: %s seconds ---" % (time.time() - start_time))

In [11]:
def writeFirstEvents(size='all'):
    """
    this creates then drops duplicates and writes them to file in S3
    """
    start_time = time.time()
    # remove self references
    df = readCheckpoint(size=size).cache()
    # get first connect events, create file features, label malicious events, filter to only hosts with malicious events
    df_first_events = getFirstEvents(df).cache() 
    df_first_events.limit(1).toPandas()
    # filter first event hosts to only those with malicious events
    mal_hosts = list(df_first_events.select('hostname').filter( col('malicious') == 1 ).distinct().toPandas()['hostname'])
    df_first_events.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/first_events")
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and write time: %s seconds ---" % (time.time() - start_time))

In [15]:
def just_write_new_features(df):
    start_time = time.time()
    df.withColumn("image_path", getFileUDF(col("image_path"))) \
        .withColumn("parent_image_path", getFileUDF(col("parent_image_path"))) \
        .withColumn("new_path", getFileUDF(col("new_path"))) \
        .withColumn("file_path", getFileUDF(col("file_path"))) \
        .withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0))
    df.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/first_events")
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and write time: %s seconds ---" % (time.time() - start_time))

In [14]:
def getFile(str):
    """
    udf to get the file from a full file path
    similar udf (non-windows): https://stackoverflow.com/questions/40848681/udf-to-extract-only-the-file-name-from-path-in-spark-sql
    """
    if str == None:
        pass
    else:
        new_str = ntpath.basename(str)
        return new_str
getFileUDF = udf(lambda z: getFile(z),StringType())

In [13]:
def readFirstEvents():
    start_time = time.time()
    df = spark.read.parquet(f"{s3_url_trusted}/prod/final_features")
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read time: %s seconds ---" % (time.time() - start_time))
    return df

In [14]:
start_time = time.time()
# remove self references
df = readCheckpoint(size='all').cache()

+---------+---------+
|malicious|    count|
+---------+---------+
|        1|   188448|
|        0|452586026|
+---------+---------+



 7:40PM UTC on Mar 22, 2023 --- read and cache time: 395.0087320804596 seconds ---


In [15]:
df.count()

4595692

In [18]:
df.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/final_features")
df.unpersist()
print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and write time: %s seconds ---" % (time.time() - start_time))

 7:47PM UTC on Mar 22, 2023 --- read and write time: 782.0313174724579 seconds ---


In [19]:
start_time = time.time()
df = spark.read.parquet(f"{s3_url_trusted}/prod/final_features").cache()

In [20]:
df.groupBy('malicious').count().show()

+---------+-------+
|malicious|  count|
+---------+-------+
|        1|   8596|
|        0|4587096|
+---------+-------+



In [21]:
print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and cache time: %s seconds ---" % (time.time() - start_time))

 7:47PM UTC on Mar 22, 2023 --- read and cache time: 6.256082057952881 seconds ---


In [22]:
print(len(malcious_events))

292264


In [23]:
292264-188448

103816

In [10]:
#re-writing for testing
df = readCheckpoint(size='all').cache()

+---------+---------+
|malicious|    count|
+---------+---------+
|        1|   188448|
|        0|452586026|
+---------+---------+



 3:23PM UTC on Mar 25, 2023 --- read and cache time: 703.2229986190796 seconds ---


In [16]:
#now I want to write to first events with all the cool new features
just_write_new_features(df)

 3:50PM UTC on Mar 25, 2023 --- read and write time: 1404.1512756347656 seconds ---
